In [1]:
import ipywidgets as widgets
from IPython.display import display

In [2]:
cities_selection = widgets.RadioButtons(
    options=['presets', 'manual'],
   value='presets', 
    description='Selection:',
    disabled=False
)

In [3]:
from citySets.city_sets import city_sets

first_dropdown = widgets.Dropdown(
    options=city_sets,
    description='First region:',
)

second_dropdown = widgets.Dropdown(
    options=city_sets,
    description='Second region:',
)

first_select_display = widgets.Select(
    options=[''],
    value='',
    rows=10,
    description='Cities:',
    disabled=False
)

second_select_display = widgets.Select(
    options=[''],
    value='',
    rows=10,
    description='Cities:',
    disabled=False
)

left_box = widgets.HBox([first_dropdown, second_dropdown])
right_box = widgets.HBox([first_select_display, second_select_display])
preset_box = widgets.VBox([left_box, right_box])



def on_first_button_clicked(b):
    first_dropdown.layout.display = 'none'
    second_dropdown.layout.display = 'none'
    
# def on_second_button_clicked(b):
#     second_list.extend(dropdown.value)
#     print(second_list)

# first_button.on_click(on_first_button_clicked)
# second_button.on_click(on_second_button_clicked)



dl = widgets.dlink((first_dropdown, 'value'), (first_select_display, 'options'))
dl = widgets.dlink((second_dropdown, 'value'), (second_select_display, 'options'))


widgets.Text(
    value='',
    placeholder='Type city name',
    description='City:',
    disabled=False
)
first_button = widgets.Button(description="Add to first list")
second_button = widgets.Button(description="Add to second list")

first_manual_select_display = widgets.Select(
    options=[''],
    value='',
    rows=10,
    description='Cities:',
    disabled=False
)

second_manual_select_display = widgets.Select(
    options=[''],
    value='',
    rows=10,
    description='Cities:',
    disabled=False
)





In [4]:
place_name_input = widgets.Text(
    value='',
    placeholder='Type city name',
    description='City:',
    disabled=False
)
first_button = widgets.Button(description="Add to first list")
second_button = widgets.Button(description="Add to second list")



first_manual_select_display = widgets.Select(
    options=[],
    value=None,
    rows=10,
    description='Cities:',
    disabled=False
)

second_manual_select_display = widgets.Select(
    options=[],
    value=None,
    rows=10,
    description='Cities:',
    disabled=False
)

def add_to_first_list(b):
    first_manual_select_display.options = first_manual_select_display.options + (place_name_input.value,)
    
def add_to_second_list(b):
    second_manual_select_display.options = second_manual_select_display.options + (place_name_input.value,)    

first_button.on_click(add_to_first_list)
second_button.on_click(add_to_second_list)

left_box = widgets.HBox([first_dropdown, second_dropdown])
right_box = widgets.HBox([first_select_display, second_select_display])

first_remove_button = widgets.Button(description="Remove")
second_remove_button = widgets.Button(description="Remove")

def remove_from_list(select):
    def remove_from_list_fn(b):
        select.options = (opt for opt in select.options if opt != select.value)              
    return remove_from_list_fn

first_remove_button.on_click(remove_from_list(first_manual_select_display))
second_remove_button.on_click(remove_from_list(second_manual_select_display))

left_select_with_delete_box = widgets.VBox([first_manual_select_display, first_remove_button])
right_select_with_delete_box = widgets.VBox([second_manual_select_display, second_remove_button])

manual_input_box = widgets.HBox([place_name_input, first_button, second_button])
manual_selected_values_box = widgets.HBox([left_select_with_delete_box, right_select_with_delete_box])

preset_box = widgets.VBox([left_box, right_box])
manual_box = widgets.VBox([manual_input_box, manual_selected_values_box])


In [5]:
display(cities_selection)
display(manual_box)
display(preset_box)

manual_box.layout.display = 'none'

def on_selection_type_button_clicked(b):
    if cities_selection.value == 'presets':
        preset_box.layout.display = 'block'
        manual_box.layout.display = 'none'
    else:
        preset_box.layout.display = 'none'
        manual_box.layout.display = 'block'

cities_selection.observe(on_selection_type_button_clicked, 'value')

RadioButtons(description='Selection:', options=('presets', 'manual'), value='presets')

In [6]:
from experiments import profiles

criteria = profiles + [('All', 'all')]
profile_dropdown = widgets.Dropdown(
    options=criteria,
    description='Compare:',
)
display(profile_dropdown)

Dropdown(description='Compare:', options=(('Streets', 'streets'), ('Buildings', 'buildings'), ('Public transpo…

In [9]:
import sys
sys.path.append('../')
from run import run
from experiments import get_scores_and_similarity, get_scores, get_criteria
from dataPresentation.plotting import plot_points_with_coloring, plot_points_1_dim_with_coloring
from similarity import find_columns_with_best_silhouette_score, find_individual_columns_with_best_silhouette_score
from util.processing import remove_outliers
from pca import calculate_pca
import matplotlib.pyplot as plt
from IPython.display import display
from IPython.display import clear_output
from ipywidgets import Button, Layout
from datetime import datetime
import seaborn as sns
import pandas as pd
import os
import json

out = widgets.Output()

df = None
df_to_filter = None
df_to_display = None
checkboxes = None
chosen_profile = None
removed_columns = []

def on_column_selected(v):
    chosen_column = v.owner.description
    global df_to_display
    global df_to_filter
    global df
    global removed_columns

    if v.new == True and chosen_column in removed_columns:
        removed_columns.remove(chosen_column)
        df_to_display = df[[c for c in df_to_filter.columns if c not in removed_columns]]
    else:
        removed_columns.append(chosen_column)
        df_to_display = df[[c for c in df_to_filter.columns if c not in removed_columns]]
    display_df_as_table()
    
def select_all(v):
    global df_to_display
    global df_to_filter
    global df
    global removed_columns

    if len(removed_columns) == 0:
        removed_columns = [a for a in df_to_filter.columns]
    else:
        removed_columns.clear()
        
    df_to_display = df[[c for c in df_to_filter.columns if c not in removed_columns]]
    display_df_as_table()


def create_checkbox(description, value=True):
    cb = widgets.Checkbox(
            value=value,
            description=description,
            disabled=False,
            indent=False)
        
    cb.observe(on_column_selected, names='value')
    return cb


def display_df_as_table(b=None):
    global df_to_display
    global df_to_filter
    global checkboxes
    global removed_columns
    checkboxes = [create_checkbox(column, column not in removed_columns) for column in df_to_filter.columns]
    with(out):
        clear_output(wait=True)
        cm = sns.light_palette("green", as_cmap=True)
        df_styled = df_to_display.style.background_gradient(cmap=cm)
        display(df_styled)
        checkboxes_panel = widgets.HBox(checkboxes, layout=Layout(width='55%'))
        display(checkboxes_panel)



def refresh_df_to_display_with_profile():
    global df_to_display
    global df
    global df_to_filter

    global chosen_profile
    if df is not None:
        fn_names = [fn.original_func_name for fn in get_criteria(chosen_profile)]
        df_to_display = df[[fn_name for fn_name in fn_names if fn_name in df.columns]]
        df_to_filter = df[[fn_name for fn_name in fn_names if fn_name in df.columns]]
        display_df_as_table()
        if not all([fn_name in df.columns for fn_name in fn_names]):
            with(out):
                print('NOT ALL CRITERIA PRESENT FROM THE PROFILE' + str([fn_name for fn_name in fn_names if fn_name not in df.columns]))
        
def on_profile_chosen(v):
    global chosen_profile
    global removed_columns
    removed_columns = []
    chosen_profile = v.new
    refresh_df_to_display_with_profile()

def on_remove_outliers_selected(v):
    global df_to_display
    if v.new is True:
        df_to_display = df_to_display.dropna()
        df_to_display = remove_outliers(df_to_display)
        print(df_to_display.index)
        display_df_as_table()
    else:
        refresh_df_to_display_with_profile()
        display_df_as_table()

remove_outliers_checkbox_for_calculations = create_checkbox('Remove outliers', False)

            
profile_display_chooser_dropdown = widgets.Dropdown(
    options=criteria,
    description='Profile:',
)

profile_display_chooser_dropdown.observe(on_profile_chosen, names='value')
remove_outliers_checkbox_for_calculations.observe(on_remove_outliers_selected, names='value')

first_group = widgets.Text(
    value=None,
    description='First group:',
    disabled=False
)

second_group = widgets.Text(
    value=None,
    description='Second group:',
    disabled=False
)

save_to_csv_path = widgets.Text(
    value=None,
    description='Path:',
    disabled=False
)    

read_csv_path = widgets.Text(
    value=None,
    description='Path:',
    disabled=False
)        

def read_file_to_dataframe(b):
    global df
    global df_to_display
    global df_to_filter

    df = pd.read_csv(read_csv_path.value, index_col=0)
    df_to_display=df.copy()
    df_to_filter=df.copy()


def get_left_select_values():
    if first_group.value != "":
        left_values = json.loads(first_group.value)
        return (left_values['region'], left_values['cities'])
    else:
        calculated_in_nb_values = list(first_manual_select_display.options) if cities_selection.value == 'manual' else list(first_select_display.options)
        return ('First region', calculated_in_nb_values)
        
def get_right_select_values():
    right_values = None
    if second_group.value != "":
        right_values = json.loads(second_group.value)
        return (right_values['region'], right_values['cities'])
    else: 
        calculated_in_nb_values = list(second_manual_select_display.options) if cities_selection.value == 'manual' else list(second_select_display.options)
        return ('Second region', calculated_in_nb_values)

def run_scores(b):
    global df
    global df_to_display
    values = get_left_select_values()[1] + get_right_select_values()[1]
    df = get_scores(values, [profile_dropdown.value])
    df_to_display=df.copy()
    display_df_as_table()


no_of_columns_slider = widgets.IntSlider(
    value=2,
    min=2,
    max=6,
    step=1,
    description='No of columns:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='d'
)    
    
def display_best_silhouette_score(b):
    silhouette_df = scale_min_max(df_to_display.dropna())
    with(out):
        print(find_columns_with_best_silhouette_score(silhouette_df.loc[get_left_select_values()[1]], silhouette_df.loc[get_right_select_values()[1]], no_of_columns_slider.value))
        
def display_best_individual_silhouette_score(b):
    silhouette_df = scale_min_max(df_to_display.dropna())
    with(out):
        print(find_individual_columns_with_best_silhouette_score(silhouette_df.loc[get_left_select_values()[1]], silhouette_df.loc[get_right_select_values()[1]], no_of_columns_slider.value))
        
def save_to_csv(b):
    global df
    file_path_no_format = save_to_csv_path.value.split('.')[0]
    df.to_csv('out/' + file_path_no_format + '.csv')
    
calculate_button = widgets.Button(description="Calculate paremeters")
save_to_csv_button = widgets.Button(description="Save to CSV")
read_from_csv_button = widgets.Button(description="Read from CSV")
display_csv_button = widgets.Button(description="Display")
best_columns_button = widgets.Button(description="Find best columns combination")
best_individual_columns_button = widgets.Button(description="Find best individual columns")


select_all_button = widgets.Button(description="Select/Deselect all")


calculate_button.on_click(run_scores)
display_csv_button.on_click(display_df_as_table)
read_from_csv_button.on_click(read_file_to_dataframe)
save_to_csv_button.on_click(save_to_csv)
select_all_button.on_click(select_all)
best_columns_button.on_click(display_best_silhouette_score)
best_individual_columns_button.on_click(display_best_individual_silhouette_score)

read_from_csv_box = widgets.Box([read_csv_path, read_from_csv_button, display_csv_button, no_of_columns_slider, best_individual_columns_button, best_columns_button,  select_all_button])
csv_box_with_groups = widgets.VBox([first_group, second_group, read_from_csv_box])

calculate_and_save_box = widgets.VBox([calculate_button, profile_display_chooser_dropdown, remove_outliers_checkbox_for_calculations, save_to_csv_path, save_to_csv_button])


display(widgets.HBox([calculate_and_save_box, csv_box_with_groups]), out)



Output()

In [8]:
from coloringSimilarity.draw_boundary import display_similarity_on_map2, extract_osm_id
from most_similar import find_most_similar_in_df
from plots import plot_rows_from_df
from similarity import scale_min_max
import numpy as np
from ipywidgets import Label
from similarity import silhouette_score

out2 = widgets.Output()

scale_cities_input = widgets.FloatText(
    value=3,
    description='Scale cities by:',
    disabled=False
)        
     
country_to_draw = widgets.Text(
    value='Poland',
    description='Country to draw',
    disabled=False
)        

remove_outliers_checkbox = create_checkbox('Remove outliers', False)
remove_outliers_most_similar_checkbox = create_checkbox('Remove outliers', False)

similarity_metric_dropdown = widgets.Dropdown(
    options=['euclidean', 'pearson'],
    description='Metric: ',
)

def draw_on_map(b):
    columns = [checkbox.description for checkbox in checkboxes if checkbox.value == True]
    try:
        pca_n1 = calculate_pca(df_to_display[columns], n_components=1, remove_outliers=remove_outliers_checkbox.value)
        pca_n2 = calculate_pca(df_to_display[columns], n_components=2, remove_outliers=remove_outliers_checkbox.value)
    except Exception as e:
        with(out2):
            print(str(e))
        return
    with(out2):
        if pca_n1.empty or pca_n2.empty:
            display("Dataframe empty after Nan values or (and) outliers were removed")
            return
        if len(pca_n1) == 1 or len(pca_n2) == 1:
            display("Need at least 1 row for PCA calculation (after Nan and outliers removed)")
            return
        clear_output(wait=True)
        df_silhouette = scale_min_max(df_to_display.dropna())
        print("Silhouette score: " + str(silhouette_score(df_silhouette[columns].loc[get_left_select_values()[1]], df_silhouette[columns].loc[get_right_select_values()[1]])))
        plt.rcParams["figure.dpi"] = 80

        pca_n2['cluster'] = [find_label(place) for place in pca_n2.index]
        plot_points_with_coloring(pca_n2)
        plot_points_1_dim_with_coloring(pca_n1)
        
        plt.rcParams["figure.dpi"] = 500
        display_similarity_on_map2(pca_n1, country_to_draw.value.split(','), 3857, linewidth=0.04, scale_cities=scale_cities_input.value, cmap='cool')
        plt.show()

def validate_all_rows_are_present_in_city_list(places):
    cities_present_in_list = all([city in places for city in df_to_display.index])
    if not all(cities_present_in_list):
        raise Exception(str(df_to_display[[not city for city in cities_present_in_list]]) + " from df missing from places list") 
        
def find_label(place):
    if extract_osm_id(place) in get_left_select_values()[1]:
        return get_left_select_values()[0]
    elif extract_osm_id(place) in get_right_select_values()[1]:
        return get_right_select_values()[0]
    else:
        raise Exception("Place not found in list: " + place)
    
def draw_2d_chart(df_to_plot, most_similar_cities):
    clear_output(wait=True)
    plt.rcParams["figure.dpi"] = 200
    plt.rcParams["legend.fontsize"] = 8
    df_to_plot = df_to_plot.rename(lambda a: a.replace('_', '\n'), axis='columns')
    scale_min_max(df_to_plot).loc[most_similar_cities, :].T.plot(fontsize=6)
    plt.xticks(range(len(df_to_plot.columns)), df_to_plot.columns)
    plt.show()

def draw_x_y_chart(df_to_plot, most_similar_cities):
    plt.rcParams["figure.dpi"] = 200
    df_to_plot = df_to_plot.rename(lambda a: a.replace('_', '\n'), axis='columns')
    scale_min_max(df_to_plot).loc[[most_similar_cities[0], most_similar_cities[1]], :].T.plot(x=most_similar_cities[0], y=most_similar_cities[1], kind='scatter', fontsize=6)
    plt.show()
    
def draw_charts(df_to_plot, most_similar_cities):
    draw_2d_chart(df_to_plot, most_similar_cities)
    draw_x_y_chart(df_to_plot, most_similar_cities)
        
def find_most_similar(b):
    with(out2):
        columns = [checkbox.description for checkbox in checkboxes if checkbox.value == True]
        df_filtered = df_to_display[columns]
        most_similar_cities, score = find_most_similar_in_df([get_left_select_values()[1], get_right_select_values()[1]], df_filtered, method=similarity_metric_dropdown.value, should_remove_outliers=remove_outliers_most_similar_checkbox.value)
        print(str(most_similar_cities) + ", " + str(score))
        draw_charts(df_filtered, most_similar_cities)

visualize_results_label = Label(value = r'\(\textbf {Visualize results}\)')
find_most_similar_label = Label(value = r'\(\textbf {Find most similar}\)')
        
draw_button = widgets.Button(description="Draw charts")
most_similar_button = widgets.Button(description="Find most similar")


draw_button.on_click(draw_on_map)
most_similar_button.on_click(find_most_similar)

data_visualization_box = widgets.VBox([visualize_results_label, scale_cities_input,country_to_draw, remove_outliers_checkbox,  draw_button])
most_similar_box = widgets.VBox([find_most_similar_label, remove_outliers_most_similar_checkbox, similarity_metric_dropdown, most_similar_button], layout=Layout(margin='0 0 0 140px'))

presentation_box = widgets.HBox([data_visualization_box, most_similar_box])

display(presentation_box, out2)
print("Note: any row with a NaN value will be dropped when drawing charts")

Output()

Note: any row with a NaN value will be dropped when drawing charts
